This notebook uses a SQLite datebase to explore the results from Synthea. The tables are described in the [CSV File Data Dictionary](https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary). Note that SQLite supports most modern SQL features, including [Common Table Expressions](https://sqlite.org/lang_with.html) and [window functions](https://sqlite.org/windowfunctions.html), as shown in the example below. Using the Pandas interface makes it easy to fetch and display the results.

In [19]:
import pandas as pd
import os
import sqlite3

CSV_DIR = 'C:/Users/rhorton/Downloads/2021_06_04T16_36_59Z/'
DB_FILE = "Missouri_10k.sqlite3"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# I include the loading function here in case you want to see how it was done.
def load_database_from_csv(db_file, csv_dir):
    db = sqlite3.connect(db_file)
    for csv_file in os.listdir(csv_dir):
        table_name = csv_file.replace('.csv', '')
        pd.read_csv(csv_dir + csv_file).to_sql(table_name, db, index=False, if_exists='replace')
    db.close()

# load_database_from_csv(db_file=DB_FILE, csv_dir=CSV_DIR)

In [20]:
db = sqlite3.connect(DB_FILE)

pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", db)

,name
0,allergies
1,careplans
2,conditions
3,devices
4,encounters
5,imaging_studies
6,immunizations
7,medications
8,observations
9,organizations


In [21]:

# This query finds the top 3 conditions for each encounter class, using CTEs and a window function.

Q = """
with
ed1 as (
    select e.encounterclass, c.description condition_description, count(*) tally
        from encounters e
        join conditions c on c.encounter = e.id
        group by encounterclass, c.description 
        order by tally desc
),
ed2 as (
    select *, 
        row_number () over ( 
          partition by encounterclass
          order by tally desc
        ) row_num
    from ed1
)
select encounterclass, condition_description, tally from ed2 where row_num <= 3;

"""

pd.read_sql_query(Q, db)

,encounterclass,condition_description,tally
0,ambulatory,Viral sinusitis (disorder),12695
1,ambulatory,Suspected COVID-19,9036
2,ambulatory,COVID-19,8730
3,emergency,Sprain of ankle,1177
4,emergency,Stroke,778
5,emergency,Drug overdose,760
6,inpatient,Hypoxemia (disorder),1795
7,inpatient,Respiratory distress (finding),1795
8,inpatient,Pneumonia (disorder),1792
9,outpatient,Otitis media,2067


In [77]:
Q = """

-- select encounterclass, count(*) tally from encounters group by encounterclass order by tally desc;
-- select description, count(*) tally from conditions group by description order by tally desc;


-- select code, description, count(*) tally from medications where lower(description) like '%insulin%' group by code, description order by tally desc

-- select code, description, count(*) tally from procedures where lower(description) like '%amput%'
--   group by code, description order by tally desc

-- select * from careplans where description like '%dialysis%' AND stop is null

-- select count(distinct patient) from careplans where code = '736690008'

-- select patient, code from conditions where code in ('422034002', '1551000119108', '97331000119101', '1501000119109')
--    order by patient, code

-- select * from observations where lower(description) like '%mass%'

-- select conditions.patient, conditions.code condition, careplans.code careplan from conditions 
--     join careplans on conditions.patient = careplans.patient
--     where conditions.code = '422034002'
--     and careplans.code = '736690008'

-- select count(distinct patient) num_patients from careplans where code = '736690008'
--     and patient not in (select distinct patient from conditions where code = '422034002')  -- 19


select count(distinct patient) num_patients from careplans where code = '736690008'
    and patient in (select distinct patient from conditions where code = '422034002')  -- 4

"""

pd.read_sql_query(Q + ' limit 100', db)

,num_patients
0,4


In [17]:
db.close()